# 1. SETTING
- 환경 설정 및 데이터 URL 불러오기

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
key = '' # 개인정보이므로 삭제했습니다.
myURI = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/TbVwEntertainment/1/1000/ko/' # 한국어

In [ ]:
print(myURI)

# 2. GET OPENAPI

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

def get_openapi(url):
  global df # 전역 변수 설정

  html = urlopen(url)
  bs = BeautifulSoup(html.read(), 'html.parser') # html.parser보다 더 깨끗한 결과 제공, 한글 스트링 호환

  itemlist = bs.find_all("row")

  df = pd.DataFrame() # 데이터를 저장할 빈 dataframe 생성

  for t in itemlist:
    try:
      language = t.find('lang_code_id').get_text()
      name = t.find('post_sj').get_text()
      address = t.find('address').get_text()
      new_address = t.find('new_address').get_text()
      tel = t.find('cmmn_telno').get_text()
      web_url = t.find('cmmn_hmpg_url').get_text()
      use_time = t.find('cmmn_use_time').get_text()
      op_info = t.find('cmmn_bsnde').get_text()
      rest_info = t.find('cmmn_rstde').get_text()
      traffic_info = t.find('subway_info').get_text()
      wep_lang = t.find('cmmn_hmpg_lang').get_text()
      stroller = t.find('cmmn_bybgy_lend_at').get_text()
    
    except: # 제공되지 않는(비어있는) 항목일 경우, 오류없이 ""로 받아오기 
      language = ""
      name = ""
      address = ""
      new_address = ""
      tel = ""
      web_url = ""
      use_time = ""
      op_info = ""
      rest_info = ""
      traffic_info = ""
      wep_lang = ""
      stroller = ""
    
    temp = pd.DataFrame(([[language, name, address, new_address, tel, web_url, use_time, op_info, rest_info, traffic_info, wep_lang, stroller]]), 
                        columns = ["언어", "상호명", "주소", "신주소", "전화번호", "웹사이트", 
                                    "운영시간", "운영요일", "휴무일", "교통정보", "홈페이지운영언어", "유모차대여여부"])
    df = pd.concat([df,temp])

  df = df.reset_index(drop=True)
  return df

In [ ]:
get_openapi(myURI)
seoul_entertainment_df_ko = df # 한국어

In [ ]:
get_openapi('http://openapi.seoul.go.kr:8088/6479696652636a673434436e57746a/xml/TbVwEntertainment/1/1000/en/') # 영어
seoul_entertainment_df_en = df

In [ ]:
get_openapi('http://openapi.seoul.go.kr:8088/6479696652636a673434436e57746a/xml/TbVwEntertainment/1/1000/ja/') # 일본어
seoul_entertainment_df_ja = df

In [ ]:
get_openapi('http://openapi.seoul.go.kr:8088/6479696652636a673434436e57746a/xml/TbVwEntertainment/1/1000/zh-CN/') # 중국어
seoul_entertainment_df_CN = df

In [ ]:
# 4개 국어 데이터 합치기
seoul_entertainment_df = pd.concat([seoul_entertainment_df_ko,seoul_entertainment_df_en, 
                                    seoul_entertainment_df_ja, seoul_entertainment_df_CN], ignore_index=True)

# 3. PREPROCESSING
  1. 언어 칼럼의 데이터를 한글로 대치 (ex. ko -> 한국어)
  2. 언어별 구 추출

In [ ]:
# 언어에 따라 데이터 대치 (ex. ko -> 한국어)
for i in range(len(seoul_entertainment_df)):
  if seoul_entertainment_df['언어'][i] == '영어':
    seoul_entertainment_df['언어'][i] = 'ENGLISH'
  elif seoul_entertainment_df['언어'][i] == '일본어':
    seoul_entertainment_df['언어'][i] = '일본어'
  elif seoul_entertainment_df['언어'][i] == '중국어':
    seoul_entertainment_df['언어'][i] = '中国人'

seoul_entertainment_df.head(5)

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보,홈페이지운영언어,유모차대여여부
0,한국어,SM 사옥,135-956 서울 강남구 청담동 52-1번지,06012 서울특별시 강남구 삼성로 648,02-6240-9800,http://www.smtown.com,,,,지하철 7호선 청담역 6번출구,,
1,한국어,롯데월드 아이스링크,138-721 서울 송파구 잠실동 40-1,05554 서울 송파구 올림픽로 240 (롯데월드),02-1661-2000,http://www.lotteworld.com/icerink/,12:00 - 20:00,,,2호선 잠실역 4번 출구 연결,,대여 가능
2,한국어,아름지기,110-040 서울 종로구 통의동 35-33,03044 서울특별시 종로구 효자로 17 (통의동),02-741-8373,http://www.arumjigi.org,AM 10:00 ~ PM 05:00,월~금,"주말, 공휴일",3호선 경복궁 3번 4번 출구,,
3,한국어,KT square (KT스퀘어),서울 종로구 세종로 100 KT광화문지사 1층,"03154 서울 종로구 세종대로 178 (세종로, KT 광화문 빌딩 West)",1577-5599,https://corp.kt.com/html/promote/culture/cultu...,평일 09:00 ~ 20:00,월~금,주말,3호선 경복궁 6번출구 세종로방향 세종문화회관 맞은편 KT광화문지사\r\n5호선 광...,,
4,한국어,금현국악원,서울 종로구 안국동 175-61번지,서울특별시 종로구 윤보선길 27 (안국동),02-723-8585,http://www.wonjh.com,개인 사정에 의해 이용 시간 변동 가능,,,3호선 안국역 1번 출구 도보 10분,,


In [ ]:
# 주소에서 구 추출
seoul_entertainment_df['구'] = ''


for i in range(len(seoul_entertainment_df)):
  # '주소' 칼럼을 기준으로 하되, 비어있으면 '신주소' 칼럼을 이용해서 전체 주소 불러오기
  full_address = seoul_entertainment_df['주소'][i]
  if full_address == ' ':
    full_address = seoul_entertainment_df['신주소'][i]


  if seoul_entertainment_df['언어'][i] == '한국어': # 한국어 주소 추출
    try:
      seoul_entertainment_df['구'][i] = full_address.split(' ')[2]
      if seoul_entertainment_df['구'][i][-1] != '구':
        seoul_entertainment_df['구'][i] = 'error'
    except:
      seoul_entertainment_df['구'][i] = 'error'

  elif seoul_entertainment_df['언어'][i] == '영어': # 영어 주소 추출
    try:
      address = seoul_entertainment_df['주소'][i].split(',')
      seoul_entertainment_df['구'][i] = address[['-gu' in address[i] for i in range(len(address))].index(True)].replace(' ', '')
    except:
      seoul_entertainment_df['구'][i] = 'error'

  elif seoul_entertainment_df['언어'][i] == '일본어': # 일본어 주소 추출
    try:
      address1 = full_address.split('市')
      address2 = address1[1].split('区')
      seoul_entertainment_df['구'][i] = address2[0]+'区'
    except:
      seoul_entertainment_df['구'][i] = 'error'

  else: # 중국어 주소 추출
    if '市' in full_address:
      address1 = full_address.split('市')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_entertainment_df['구'][i] = address2[0] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_entertainment_df['구'][i] = address2[0] + '区'
      else:
        seoul_entertainment_df['구'][i] = 'error'
      
    elif '首' in full_address:
      address1 = full_address.split('首')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_entertainment_df['구'][i] = address2[0][1:] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_entertainment_df['구'][i] = address2[0][1:] + '区'
      else:
        seoul_entertainment_df['구'][i] = 'error'

In [ ]:
seoul_entertainment_df

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보,홈페이지운영언어,유모차대여여부,구
0,한국어,SM 사옥,135-956 서울 강남구 청담동 52-1번지,06012 서울특별시 강남구 삼성로 648,02-6240-9800,http://www.smtown.com,,,,지하철 7호선 청담역 6번출구,,,강남구
1,한국어,롯데월드 아이스링크,138-721 서울 송파구 잠실동 40-1,05554 서울 송파구 올림픽로 240 (롯데월드),02-1661-2000,http://www.lotteworld.com/icerink/,12:00 - 20:00,,,2호선 잠실역 4번 출구 연결,,대여 가능,송파구
2,한국어,아름지기,110-040 서울 종로구 통의동 35-33,03044 서울특별시 종로구 효자로 17 (통의동),02-741-8373,http://www.arumjigi.org,AM 10:00 ~ PM 05:00,월~금,"주말, 공휴일",3호선 경복궁 3번 4번 출구,,,종로구
3,한국어,KT square (KT스퀘어),서울 종로구 세종로 100 KT광화문지사 1층,"03154 서울 종로구 세종대로 178 (세종로, KT 광화문 빌딩 West)",1577-5599,https://corp.kt.com/html/promote/culture/cultu...,평일 09:00 ~ 20:00,월~금,주말,3호선 경복궁 6번출구 세종로방향 세종문화회관 맞은편 KT광화문지사\r\n5호선 광...,,,종로구
4,한국어,금현국악원,서울 종로구 안국동 175-61번지,서울특별시 종로구 윤보선길 27 (안국동),02-723-8585,http://www.wonjh.com,개인 사정에 의해 이용 시간 변동 가능,,,3호선 안국역 1번 출구 도보 10분,,,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,중국어,ECC(Ewha Campus Complex),120-750 首尔市西大门区大岘洞11-1,03760 首尔市西大门区梨花女大街52 (大岘洞、梨大校园内),,,,,,2号线 梨大站3号出口,,,西大门区
828,중국어,延世大学新村校区,120-749 首尔市西大门区新村洞134,03722 首尔市西大门区延世路50 (新村洞),02-2123-2114,https://www.yonsei.ac.kr/sc/index.jsp,,,,2号线 新村站2、3号出口,韩语、英语、中文,,西大门区
829,중국어,中浪青少年体验之林学习中心,131-800 首尔市中浪区忘忧洞76,02062 首尔市中浪区忘忧路87街110 (忘忧洞),+82-2-432-2567,http://www.joongrangsoop.com/forest/,各项目不同,,,京义中央线 养源站2号出口,韩语,,中浪区
830,중국어,LATTCT儿童剧场,135-854 首尔市江南区道谷洞514-1,06296 首尔市江南区论岘路164 (UNIBOOKS办公大楼),+82-2-560-0999,http://www.latt.co.kr/LattCT/Intro.aspx,10:00~17:00\r\n,周二~周日,周一,3号线 良才站5号出口步行15分钟 \r\n3号线 梅峰站4号出口步行15分钟,,,江南区


In [ ]:
seoul_entertainment_df.to_excel('/content/drive/MyDrive/빅데이터통계분석/data/seoul_entertainment.xlsx')